In [11]:
import pandas as pd
from matplotlib import pyplot
import plotly.express as px

import pymssql

# config file with credentials
from config import database
from config import username
from config import password
from config import server
from config import table

In [12]:
#dl = District Level, sl = School Level
dl_2022 = pd.read_csv('../Datasets/dl_2022.csv')
sl_2022 = pd.read_csv('../Datasets/sl_2022.csv')

In [13]:
# Cleaning/Filtering

DLML1322 = dl_2022.loc[(dl_2022['Time Period1'] == 'EOY')]

DLMLF1322 = DLML1322[
    [
        'Category',
        'School Year',
        '# of Misconducts2',
        '# of Group 1-2 (minor) Misconducts3',
        '# of Group 3-4 (moderate) Misconducts4',
        '# of Group 5-6 (major) Misconducts5',
        '# of Police Notifications',
        '% of Misconducts Resulting in Police Notification'
    ]
]

DLMLF1322 = DLMLF1322.loc[
    ((DLMLF1322['Category'] == 'Grade PK-2') 
    | (DLMLF1322['Category'] == 'Grade 3-5') 
    | (DLMLF1322['Category'] == 'Grade 6-8') 
    | (DLMLF1322['Category'] == 'Grade 9-12'))
    ]

DLMLF1322['School Year'] = DLMLF1322['School Year'].astype('category').cat.codes

In [14]:
# Correlation Heatmap
fig = px.imshow(
    DLMLF1322.corr(),
    color_continuous_scale = 'RdBu_r',
    range_color = [-1,1],
    labels=dict(color='correlation'),
    width=720,
    height=560,
    text_auto = '.2f',
    aspect='auto'
)

fig.update_xaxes(side='top')
fig.show()

In [15]:
# Scatterplot for minor incidents vs police notifications
fig = px.scatter(
    DLMLF1322,
    x = '# of Group 1-2 (minor) Misconducts3',
    y = '# of Police Notifications',
    color = 'Category'
)

fig.update_layout(title = 'Minor Misconduct Incidents vs. Number of Police Notifications')
fig.show()

In [16]:
# Scatterplot for moderate incidents vs police notifications
fig = px.scatter(
    DLMLF1322,
    x = '# of Group 3-4 (moderate) Misconducts4',
    y = '# of Police Notifications',
    color = 'Category'
)

fig.update_layout(title = 'Moderate Misconduct Incidents vs. Number of Police Notifications')
fig.show()

In [17]:
# Scatterplot for major incidents vs police notifications
fig = px.scatter(
    DLMLF1322,
    x = '# of Group 5-6 (major) Misconducts5',
    y = '# of Police Notifications',
    color = 'Category'
)

fig.update_layout(title = 'Major Misconduct Incidents vs. Number of Police Notifications')
fig.show()

In [18]:
# Extracting neighborhood info from SQL database
query = f'''
SELECT 
    community_area,
    count(community_area) AS num_crimes
FROM {table}
WHERE community_area IS NOT NULL
GROUP BY community_area
ORDER BY community_area
'''

con = pymssql.connect(server, username, password, database)

comm = pd.read_sql(query, con)

# Cleaning / Organizing
comm['community_area'] = comm['community_area'].astype('int')
comm = comm.sort_values('community_area').reset_index(drop=True)

In [21]:
wards = pd.read_csv('../Datasets/wards.csv')

school_comm = wards[['School ID', 'Community Area Number']]

In [20]:
misconduct = sl_2022[
    [
        'School ID', 
        '# of Misconducts', 
        '# of Suspensions (includes ISS and OSS)',
        '# of Students Expelled',
        '# of Police Notifications'
        ]
    ]

misconduct = misconduct[misconduct['# of Misconducts'] != '--']
misconduct = misconduct[misconduct['# of Suspensions (includes ISS and OSS)'] != '--']
misconduct = misconduct[misconduct['# of Police Notifications'].astype('int64') != '--']

In [22]:
# Merging to get crimes by community area
school_mis = misconduct.merge(
    school_comm,
    how = 'inner',
    on = 'School ID',
    )

school_df = school_mis.merge(
    comm,
    how= 'inner',
    left_on = 'Community Area Number',
    right_on = 'community_area' 
).drop(columns='Community Area Number')

# Cleaning
school_df['# of Misconducts'] = school_df['# of Misconducts'].astype('int')
school_df['# of Suspensions (includes ISS and OSS)'] = school_df['# of Suspensions (includes ISS and OSS)'].astype('int')
school_df['# of Police Notifications'] = school_df['# of Police Notifications'].astype('int')

In [23]:
# num_crimes is already aggregated so no need to sum
miscon_crimes = school_df.groupby(['community_area']) \
.agg(
    {
    '# of Misconducts': 'sum',
    '# of Suspensions (includes ISS and OSS)' : 'sum',
    '# of Students Expelled': 'sum',
    '# of Police Notifications' : 'sum',
    'num_crimes' : 'mean',
    }
)

In [24]:
# Correlation between crimes and school incidents
import plotly.express as px

fig = px.imshow(
    miscon_crimes.corr(),
    color_continuous_scale = 'deep',
    range_color = [-1,1],
    labels=dict(color='correlation'),
    width=720,
    height=560,
    text_auto = '.2f',
    aspect='auto'
)

fig.update_xaxes(side='top')
fig.show()